In [12]:
import numpy as np
import pandas as pd

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

import numpy as np
np.random.seed(2018)

First we will only work on Title

In [13]:
course_details = pd.read_pickle('Preproccessed Data/Explored Data/course_details.pkl')

In [14]:
course_syllabi = pd.read_pickle('Preproccessed Data/Explored Data/course_syllabi.pkl')

In [15]:
course_reviews = pd.read_pickle('Preproccessed Data/Explored Data/course_reviews.pkl')

In [35]:
# column for text to be processed, choose one line

course_details['source_text'] = course_details['title']
#course_details['source_text'] = course_details.apply(lambda row : row['title']+' '+''.join(row['objectives']), axis =1)
#course_details['source_text'] = course_details.apply(lambda row : row['title']+' '+''.join(row['prerequisites']), axis =1)

course_details['source_text']

id
8082                          Ruby Programming for Beginners
8203                         Basic and Advanced HTML Lessons
8319                      Git Basics: In Theory and Practice
8324                                Javascript for Beginners
8325                 HTML Tutorial: HTML & CSS for Beginners
8416       Beginners - How To Create iPhone And iPad Apps...
8723             C Programming: iOS Development Starts Here!
9463       Programming Java for Beginners - The Ultimate ...
10321                              Learn to Program with C++
10620      Practical Transformation Using XSLT and XPath ...
10629         Introduction to Microsoft SQL Server Databases
10681         Advanced Ruby Programming: 10 Steps to Mastery
11064      Learn to make HTML 5, Facebook, Chrome Store g...
11152         Learning C# .NET - Programming for Beginners. 
11153         Learn jQuery: An In-depth Course For Beginners
11174                    Become a Web Developer from Scratch
11197                

In [36]:
def lemmatize_stemming(text):
    stemmer = SnowballStemmer('english')
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

In [46]:
Negative_keywords = [
    'learning',
    'learn',
    'course',
    'complete',
    'create',
    'developing',
    
    'development',
    'programming'
    
    'coding',
    'build',
    'scratch',
    
    'getting',
    'started,'
    'easy',
    'basic',
    'basics',
    'advanced',
    'fundamentals',
    'beginners',
    
    'tutorial',
    'lessons',
    'practical',
    'depth',
    'introduction',
    'better'
    
]

In [47]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if (token not in gensim.parsing.preprocessing.STOPWORDS) and (token not in Negative_keywords):
            #result.append(lemmatize_stemming(token))
            result.append(token)
    return result

In [48]:
course_details['PreprocessLDA'] = course_details['source_text'].apply(preprocess)
course_details['PreprocessLDA'].head(100)

id
8082                                   [ruby, programming]
8203                                                [html]
8319                               [git, theory, practice]
8324                                          [javascript]
8325                                     [html, html, css]
8416                    [iphone, ipad, apps, zero, coding]
8723                            [programming, ios, starts]
9463                   [programming, java, ultimate, java]
10321                                            [program]
10620    [transformation, xslt, xpath, hour, free, prev...
10629                  [microsoft, sql, server, databases]
10681                  [ruby, programming, steps, mastery]
11064               [html, facebook, chrome, store, games]
11152                                   [net, programming]
11153                                             [jquery]
11174                                     [web, developer]
11197                                    [php, php, m

In [49]:
# Bag of words dict
dictionary = gensim.corpora.Dictionary(course_details['PreprocessLDA'])

In [50]:
# Bag of words corpus
course_details['bow_corpus'] = course_details['PreprocessLDA'].apply(lambda x : dictionary.doc2bow(x)) 
bowcorpus = course_details['bow_corpus'].values.tolist()

###### tf idf


In [51]:
from gensim import corpora, models
tfidf = models.TfidfModel(bowcorpus)
corpus_tfidf = tfidf[bowcorpus]

In [52]:
models

<module 'gensim.models' from 'C:\\Users\\Omar-Bouj\\Anaconda3\\lib\\site-packages\\gensim\\models\\__init__.py'>

### LDA with bagOfWords

In [53]:
lda_model = gensim.models.LdaMulticore(bowcorpus, num_topics=15, id2word=dictionary, passes=2, workers=2)

In [54]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.025*"ios" + 0.023*"swift" + 0.018*"java" + 0.018*"web" + 0.017*"master" + 0.015*"unity" + 0.015*"website" + 0.015*"apps" + 0.014*"android" + 0.014*"app"
Topic: 1 
Words: 0.068*"html" + 0.053*"css" + 0.022*"beginner" + 0.022*"responsive" + 0.019*"website" + 0.017*"web" + 0.013*"javascript" + 0.012*"php" + 0.011*"kotlin" + 0.010*"framework"
Topic: 2 
Words: 0.034*"game" + 0.030*"unity" + 0.017*"spring" + 0.017*"ios" + 0.015*"games" + 0.014*"android" + 0.014*"developers" + 0.014*"app" + 0.014*"building" + 0.011*"apps"
Topic: 3 
Words: 0.040*"bootstrap" + 0.032*"net" + 0.031*"es" + 0.022*"web" + 0.021*"asp" + 0.018*"core" + 0.015*"application" + 0.012*"visual" + 0.012*"react" + 0.012*"ecmascript"
Topic: 4 
Words: 0.054*"python" + 0.030*"programming" + 0.029*"android" + 0.028*"testing" + 0.022*"php" + 0.018*"object" + 0.016*"data" + 0.015*"oriented" + 0.015*"beginner" + 0.013*"java"
Topic: 5 
Words: 0.082*"java" + 0.050*"programming" + 0.040*"python" + 0.021*"application"

In [55]:
for index, score in sorted(lda_model[bowcorpus[8082]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.8444422483444214	 
Topic: 0.025*"ios" + 0.023*"swift" + 0.018*"java" + 0.018*"web" + 0.017*"master" + 0.015*"unity" + 0.015*"website" + 0.015*"apps" + 0.014*"android" + 0.014*"app"

Score: 0.011111308820545673	 
Topic: 0.055*"app" + 0.039*"data" + 0.029*"android" + 0.020*"apps" + 0.018*"ios" + 0.017*"programming" + 0.015*"mobile" + 0.014*"beginner" + 0.014*"react" + 0.014*"python"

Score: 0.011111282743513584	 
Topic: 0.034*"game" + 0.030*"unity" + 0.017*"spring" + 0.017*"ios" + 0.015*"games" + 0.014*"android" + 0.014*"developers" + 0.014*"app" + 0.014*"building" + 0.011*"apps"

Score: 0.011111272498965263	 
Topic: 0.044*"javascript" + 0.035*"css" + 0.027*"html" + 0.022*"hands" + 0.018*"hour" + 0.017*"guide" + 0.017*"machine" + 0.014*"game" + 0.014*"website" + 0.012*"swift"

Score: 0.011111272498965263	 
Topic: 0.034*"js" + 0.030*"web" + 0.019*"app" + 0.019*"sql" + 0.017*"blockchain" + 0.015*"application" + 0.015*"unity" + 0.015*"node" + 0.014*"game" + 0.012*"applications"

S

### LDA with TF IDF

In [56]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [57]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.030*"html" + 0.029*"javascript" + 0.017*"css" + 0.017*"programming" + 0.016*"java" + 0.015*"python" + 0.015*"step" + 0.015*"projects" + 0.012*"php" + 0.010*"web"
Topic: 1 
Words: 0.017*"android" + 0.016*"ios" + 0.014*"swift" + 0.013*"python" + 0.010*"apps" + 0.010*"programming" + 0.009*"game" + 0.009*"laravel" + 0.008*"building" + 0.008*"web"
Topic: 2 
Words: 0.013*"js" + 0.013*"node" + 0.012*"java" + 0.011*"spring" + 0.010*"beginning" + 0.009*"javascript" + 0.009*"algorithms" + 0.008*"structures" + 0.008*"web" + 0.007*"mongodb"
Topic: 3 
Words: 0.034*"programming" + 0.013*"app" + 0.013*"guide" + 0.013*"game" + 0.012*"android" + 0.012*"mastering" + 0.011*"jquery" + 0.011*"python" + 0.011*"unity" + 0.010*"mobile"
Topic: 4 
Words: 0.024*"unity" + 0.016*"game" + 0.012*"games" + 0.009*"js" + 0.009*"coding" + 0.007*"python" + 0.007*"beginner" + 0.007*"javascript" + 0.007*"web" + 0.006*"zero"
Topic: 5 
Words: 0.012*"guide" + 0.011*"web" + 0.011*"python" + 0.010*"typescript

In [58]:
for index, score in sorted(lda_model[bowcorpus[8082]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.8444422483444214	 
Topic: 0.025*"ios" + 0.023*"swift" + 0.018*"java" + 0.018*"web" + 0.017*"master" + 0.015*"unity" + 0.015*"website" + 0.015*"apps" + 0.014*"android" + 0.014*"app"

Score: 0.011111298575997353	 
Topic: 0.055*"app" + 0.039*"data" + 0.029*"android" + 0.020*"apps" + 0.018*"ios" + 0.017*"programming" + 0.015*"mobile" + 0.014*"beginner" + 0.014*"react" + 0.014*"python"

Score: 0.011111282743513584	 
Topic: 0.034*"game" + 0.030*"unity" + 0.017*"spring" + 0.017*"ios" + 0.015*"games" + 0.014*"android" + 0.014*"developers" + 0.014*"app" + 0.014*"building" + 0.011*"apps"

Score: 0.011111272498965263	 
Topic: 0.044*"javascript" + 0.035*"css" + 0.027*"html" + 0.022*"hands" + 0.018*"hour" + 0.017*"guide" + 0.017*"machine" + 0.014*"game" + 0.014*"website" + 0.012*"swift"

Score: 0.011111272498965263	 
Topic: 0.034*"js" + 0.030*"web" + 0.019*"app" + 0.019*"sql" + 0.017*"blockchain" + 0.015*"application" + 0.015*"unity" + 0.015*"node" + 0.014*"game" + 0.012*"applications"

S

If no improvement in Topic modelling, Topic will be choosen manually and assigned automatically

In [ ]:
Languages = """
Assembly
Bash
Shell
PowerShell
C
C++
C#
Clojure
Dart
Elixir
Erlang
F#
Go
HTML
CSS
Java
JavaScript
Kotlin
ObjectiveC
PHP
Python
R
Ruby
Rust
Scala
SQL
Swift
TypeScript
VBA
WebAssembly
Git
"""

In [ ]:
DBMS = """
Cassandra
Couchbase
DynamoDB
Elasticsearch
Firebase
MariaDB
MongoDB
Microsoft SQL Server
MySQL
Oracle
PostgreSQL
Redis
SQLite
"""

In [ ]:
Web = """
Angular/Angular.js
ASP.NET
Django
Drupal
Express
Flask
jQuery
Laravel
React.js
Ruby on Rails
Spring
Vue.js
"""

In [ ]:
Roles = """
Academic researcher
Data or business analyst
Data scientist or machine learning specialist
Database administrator
Designer
Developer, backend
Developer, desktop or enterprise applications
Developer, embedded applications or devices
Developer, frontend
Developer, fullstack
Developer, game or graphics
Developer, mobile
Developer, QA or test
DevOps specialist
Educator
Engineer, data
Engineer, site reliability
Engineering manager
Marketing or sales professional
Product manager
Scientist
Senior Executive (CSuite,VP, etc.)
Student
System administrator
"""